In [1]:
from classeg.extensions.unstable_diffusion.inference.inferer import UnstableDiffusionInferer

inferer = UnstableDiffusionInferer(
        "Dataset_large_421",
        0,
        "attention_double",
        "latest",
        None
    )

/home/student/andrewheschl/Documents/Diffusion_ClasSeg/.venv/lib/python3.11/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.20 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
inferer.pre_infer()

/home/student/andrewheschl/Documents/Diffusion_ClasSeg/classeg/extensions/unstable_diffusion/inference/inferer.py:115: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoi

'/home/student/andrewheschl/Documents/Datasets/ClassificationDataset/results/Dataset_large_421/fold_0/attention_double/inference'

In [7]:
# /home/student/andrewheschl/Documents/Datasets/ClassificationDataset/preprocessed/Dataset_large_421/fold_0/val/imagesTr
# is a folder with a bunch of .npy images.
# make a dataset out of this folder, and a dataloader out of the dataset
# code is here, not commented:

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import torch
import matplotlib.pyplot as plt
import tqdm
# use tensorboard to visualize the embeddings
from torch.utils.tensorboard import SummaryWriter


def numpy_loader(path):
    sample = np.load(path)
    return torch.from_numpy(sample).to(torch.float32)

# no classes because we're not doing classification
dataset = datasets.DatasetFolder(
    "/home/student/andrewheschl/Documents/Datasets/ClassificationDataset/preprocessed/Dataset_large_421/fold_0/val",
    loader=numpy_loader,
    transform=transforms.Compose([
        transforms.RandomCrop((512, 512)),
        # cv2.INTER_CUBIC
        transforms.Resize((128, 128), interpolation=transforms.InterpolationMode.BICUBIC),
    ]),
    # classes=['imagesTr'],
    is_valid_file=lambda x: "imagesTr" in x
)

dataloader = DataLoader(dataset, batch_size=64, shuffle=False, pin_memory=True)
inferer.model.eval()
inferer.model = inferer.model.cuda()

embeddings = []
images_all = []
for images, labels in tqdm.tqdm(dataloader):
    images = images.to('cuda')
    embed, _ = inferer.model.embed_image(images)
    embed = embed.detach().cpu().numpy()
    images = images.detach().cpu().numpy()
    embeddings.append(embed)
    images_all.append(images)


  0%|          | 0/103 [00:00<?, ?it/s]

100%|██████████| 103/103 [00:25<00:00,  4.01it/s]


In [16]:
stacked_embeddings = np.vstack(embeddings)
stacked_images = torch.from_numpy(np.vstack(images_all))
# downsize the images to 64 64
writer = SummaryWriter()

from torch.nn.functional import interpolate
stacked_images = interpolate(stacked_images, (16, 16), mode='bilinear', align_corners=False)

print(stacked_embeddings.shape, stacked_images.shape)

writer.add_embedding(stacked_embeddings, label_img=stacked_images)

(6583, 512) torch.Size([6583, 3, 16, 16])
